# Iteration 4_INFOSYS 722

In [2]:
# This part starts Data Preparation of Iteration 4
# I will number each relevant cell as answer

In [3]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('iteration4').getOrCreate()
from pyspark.ml.classification import LogisticRegression

In [4]:
# Import dataset 
dataFrame = spark.read.csv('Dataset/data.csv',header=True,inferSchema=True)

In [5]:
# Show the dataset
dataFrame.show()

+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|         ANZSOCGroup|       MapDetailName|          Region|NumberofVictimisations|NumberofRecords|OccurrenceDayOfWeek|       Weapon|YearMonth|
+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|AggravatedSexualA...|      HillcrestWest.|   WaikatoRegion|                     1|              1|            UNKNOWN|NotApplicable|  1/07/14|
|Non-AggravatedSex...|   TeAwamutuCentral.|   WaikatoRegion|                     1|              1|                Sun|NotApplicable|  1/07/14|
|AggravatedSexualA...|      TotaraHeights.|  AucklandRegion|                     1|              1|                Tue|    NotStated|  1/07/14|
|AggravatedSexualA...|       FlaxmereEast.|Hawke'sBayRegion|                     1|              1|                Sun|    NotStated|  1

## 2.3 Data Exporation

In [6]:
# Computer summary of the data
dataFrame.describe().show()

# Number of AggrabatedSexualAssault report
aggSexAssuaultTable = dataFrame[dataFrame.ANZSOCGroup.isin("AggravatedSexualAssault")]
print('\n' + "Total of aggravatedSexualAssault reports:", aggSexAssuaultTable.count())

# Number of Non-AggravatedSexualAssault report
nonAggravatedSexualAssault = dataFrame[dataFrame.ANZSOCGroup.isin("Non-AggravatedSexualAssault")]
print('\n' + "Total of Non-AggravatedSexualAssault reports:", nonAggravatedSexualAssault.count())


+-------+--------------------+-------------+---------------+----------------------+---------------+-------------------+-----------+---------+
|summary|         ANZSOCGroup|MapDetailName|         Region|NumberofVictimisations|NumberofRecords|OccurrenceDayOfWeek|     Weapon|YearMonth|
+-------+--------------------+-------------+---------------+----------------------+---------------+-------------------+-----------+---------+
|  count|                8354|         7734|           7734|                  8354|           8354|               8354|       8354|     8354|
|   mean|                null|         null|           null|    0.9547522145080201|            1.0|               null|       null|     null|
| stddev|                null|         null|           null|      0.52938282883613|            0.0|               null|       null|     null|
|    min|AggravatedSexualA...|  AbbeyCaves.| AucklandRegion|                     0|              1|                Fri|   Bludgeon|  1/01/15|
|    m

## 3 Data  Preparation
### 3.2 To clean the data, issues must be made explicit, then explicitly resolved.

In [7]:
# Show orginal columns before cleaning
print("Total data columns before cleaning:", dataFrame.count())

Total data columns before cleaning: 8354


In [8]:
# Drop null values with their rows
droppedTable = dataFrame.na.drop()

In [9]:
# Show columns after cleaning
print("Total data columns after cleaning:", droppedTable.count())

droppedTable.show()

Total data columns after cleaning: 7734
+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|         ANZSOCGroup|       MapDetailName|          Region|NumberofVictimisations|NumberofRecords|OccurrenceDayOfWeek|       Weapon|YearMonth|
+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|AggravatedSexualA...|      HillcrestWest.|   WaikatoRegion|                     1|              1|            UNKNOWN|NotApplicable|  1/07/14|
|Non-AggravatedSex...|   TeAwamutuCentral.|   WaikatoRegion|                     1|              1|                Sun|NotApplicable|  1/07/14|
|AggravatedSexualA...|      TotaraHeights.|  AucklandRegion|                     1|              1|                Tue|    NotStated|  1/07/14|
|AggravatedSexualA...|       FlaxmereEast.|Hawke'sBayRegion|                     1|             

### 3.3 Data must be appropriately constructed through the creation of new features/variables, and/or data repositories/tables. 

In [10]:
from pyspark.sql.functions import format_number, col

# Sort table by offenceType
groupOffenceType = droppedTable.sort('ANZSOCGroup', ascending = True)

# Sort table by Region 
sortedTable = groupOffenceType.sort('Region',ascending = True)

In [11]:
# Import pandas.
import pandas as pd

# Take the first twenty rows of data, and visualise.
pd.DataFrame(sortedTable.take(20), columns=sortedTable.columns)

,ANZSOCGroup,MapDetailName,Region,NumberofVictimisations,NumberofRecords,OccurrenceDayOfWeek,Weapon,YearMonth
0,AggravatedSexualAssault,MtEdenNorth.,AucklandRegion,1,1,Mon,NotStated,1/07/14
1,AggravatedSexualAssault,GreyLynnEast.,AucklandRegion,1,1,Mon,NotStated,1/11/14
2,AggravatedSexualAssault,TotaraHeights.,AucklandRegion,1,1,Tue,NotStated,1/07/14
3,AggravatedSexualAssault,Mascot.,AucklandRegion,1,1,Mon,NotStated,1/07/14
4,AggravatedSexualAssault,GraftonEast.,AucklandRegion,1,1,Mon,NotStated,1/07/14
5,AggravatedSexualAssault,RandwickPark.,AucklandRegion,1,1,Wed,NotApplicable,1/10/14
6,AggravatedSexualAssault,Wiri.,AucklandRegion,1,1,Sat,NotApplicable,1/09/14
7,AggravatedSexualAssault,Balmoral.,AucklandRegion,1,1,Thu,NotStated,1/07/14
8,AggravatedSexualAssault,StarlingPark.,AucklandRegion,1,1,Mon,NotApplicable,1/09/14
9,AggravatedSexualAssault,OtaraNorth.,AucklandRegion,2,1,Sun,NotApplicable,1/11/14


In [12]:
# Visualize the data 
sortedTable.groupby('ANZSOCGroup').count().toPandas()

,ANZSOCGroup,count
0,AggravatedSexualAssault,5657
1,Non-AggravatedSexualAssault,2077


## 7 Data Mining

In [13]:
# Import training data
sortedTable.printSchema()
print(sortedTable.columns)

root
 |-- ANZSOCGroup: string (nullable = true)
 |-- MapDetailName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- NumberofVictimisations: integer (nullable = true)
 |-- NumberofRecords: integer (nullable = true)
 |-- OccurrenceDayOfWeek: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- YearMonth: string (nullable = true)

['ANZSOCGroup', 'MapDetailName', 'Region', 'NumberofVictimisations', 'NumberofRecords', 'OccurrenceDayOfWeek', 'Weapon', 'YearMonth']


In [14]:
# Import the relevant packages.
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [15]:
# Create a string indexer which converts every string into a number
# such as AggravtedSexAssualt = 0 and Non-AggravtedSexAssualt = 1.
# A number will be assigned to every category in the column.
# Set offencetype as label
offenceType_indexer = StringIndexer(inputCol='ANZSOCGroup',outputCol='label')
areaName_indexer = StringIndexer(inputCol='MapDetailName',outputCol='areaIndex')
region_indexer = StringIndexer(inputCol='Region',outputCol='regionIndex')
occurDay_indexer = StringIndexer(inputCol='OccurrenceDayOfWeek',outputCol='occurDayIndex')
weapon_indexer = StringIndexer(inputCol='Weapon',outputCol='weaponIndex')
yearMonth_indexer = StringIndexer(inputCol='YearMonth',outputCol='yearMonthIndex')


# Conver the various outputs into a single vector.
# Multiple columns are collapsed into one. 
areaName_encoder = OneHotEncoder(inputCol='areaIndex',outputCol='areaVec')
region_encoder = OneHotEncoder(inputCol='regionIndex',outputCol='regionVec')
occurDay_encoder = OneHotEncoder(inputCol='occurDayIndex',outputCol='occurDayVec')
weapon_encoder = OneHotEncoder(inputCol='weaponIndex',outputCol='weaponVec')
yearMonth_encoder = OneHotEncoder(inputCol='yearMonthIndex',outputCol='yearMonthVec')


# Use vector assembler to turn all of these columns into one column (named features).
assembler = VectorAssembler(inputCols=['areaVec','regionVec','occurDayVec',
                                       'weaponVec','yearMonthVec','NumberofVictimisations', 
                                       'NumberofRecords'], outputCol="features")

In [16]:
from pyspark.ml import Pipeline

# Go through each columns to pipeline
pipeline = Pipeline(stages=[areaName_indexer, region_indexer, occurDay_indexer, weapon_indexer, yearMonth_indexer, 
                            areaName_encoder, region_encoder, occurDay_encoder, weapon_encoder, yearMonth_encoder,
                            assembler])

# Apply it to the data.
pipeline_model = pipeline.fit(sortedTable)

# Incorporate results into a new DataFrame.
pipe_df = pipeline_model.transform(sortedTable)

# Remove all variables other than features and label. 
pipe_df = pipe_df.select('label', 'features')

AnalysisException: "cannot resolve '`label`' given input columns: [weaponIndex, regionVec, MapDetailName, regionIndex, occurDayVec, Region, yearMonthVec, areaVec, Weapon, NumberofVictimisations, areaIndex, weaponVec, ANZSOCGroup, occurDayIndex, OccurrenceDayOfWeek, yearMonthIndex, YearMonth, features, NumberofRecords];;\n'Project ['label, features#878]\n+- Project [ANZSOCGroup#0, MapDetailName#1, Region#2, NumberofVictimisations#3, NumberofRecords#4, OccurrenceDayOfWeek#5, Weapon#6, YearMonth#7, areaIndex#721, regionIndex#732, occurDayIndex#744, weaponIndex#757, yearMonthIndex#771, areaVec#786, regionVec#802, occurDayVec#819, weaponVec#837, yearMonthVec#856, UDF(named_struct(areaVec, areaVec#786, regionVec, regionVec#802, occurDayVec, occurDayVec#819, weaponVec, weaponVec#837, yearMonthVec, yearMonthVec#856, NumberofVictimisations_double_VectorAssembler_4bc083863dfa24761a9c, cast(NumberofVictimisations#3 as double), NumberofRecords_double_VectorAssembler_4bc083863dfa24761a9c, cast(Numb